# Optimization
This notebook will cover techniques for going from "Julia code that runs" to "Julia code that runs *fast*(er)!"

Another useful "one-stop-shop" resource in this category (there are many more!) is the [Optimization post on Modern Julia Workflows](https://modernjuliaworkflows.org/optimizing/).

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using BenchmarkTools
using StaticArrays
using Random
using Plots
using LinearAlgebra

### Row/Column Major:
Julia is column-major, which means columns are contiguous in memory. This is the same as MATLAB and Fortran, but Numpy is row-major by default. Remeber this fact when looping over data as looping incorrectly comes with a big performance penalty!

### Benchmarking Code

Julia is just in time (JIT) compiled, so the first time you run code there will be a penalty to first compile the code before it can be run. We can use the `@time` macro to see this effect. (If you run the cell more than once, the effect will go away since it has already been compiled!)

In [4]:
x = rand(1000);
function sum_global()
    s = 0.0
    for i in x
        s += i
    end
    return s
end;

In [ ]:
@time sum_global();
@time sum_global();

The `@time` macro is convenient, but only executes the function once. We can use the [BenchmarkTools](https://juliaci.github.io/BenchmarkTools.jl/stable/) library to investigate the performance of our function with multiple trials. The `@benchmark` macro will run 10,000 trials or (roughly) 5 seconds whichever comes first. (These can be configured). There is also the `@btime` macro which runs the same number of trials as `@benchmark` but has output similar to `@time`.

In [ ]:
@benchmark sum_global()

If your function takes parameters, be sure to "interpolate" them with a `$` when using `@benchmark` this tells BenchmarkTools to ignore the allocation and time required to allocate the parameters.

In [ ]:
# Parameter is Interpolated
@btime inv($(rand(6,6)));
# No Interpolation
@btime inv(rand(6,6));

### Reducing Memory Allocations
One of the easiest ways to speed up your code is by removing all unnecessary memory allocations. In Python the end-user is not able to manage memory easily and the interprer or library (e.g. numpy) is trusted to handle things.

Julia can also operate like Python, where we just trust the compiler to "do the right thing". This is good for prototyping, but can result in inefficient code. Below we will look at the basics of memory allocations and how to reduce them.

-----------------------------------------------
Lets start by allocating a small array of 10,000 `Float64`s. Each `Float64` is 8 bytes (64 bits), so the total size should be 80,000 bytes or 80kB (78.125 kiB). Note there will be a slight overhead for some internal machinery associated with each allocation (e.g. a pointer to the array).

In [ ]:
@time ones(Float64, 100, 100);

In Julia, all arrays are heap-allocated since they are re-sizeable. This just means that the array is further away from the CPU and it can be slow(er) to allocate and access. The first strategy to mitigate this penalty is to simply allocate memory as little as possible. To check this we can use BenchmarkTools to understand the allocations in our function.

In [ ]:
function allocates_alot()
    res = 0.0
    for i in 1:100
        x = rand(100,100) # ALLOCATION EVERY LOOP ITERATION
        res += sum(x)
    end
    return res
end

function allocates_once()
    res = 0.0
    storage = zeros(100,100)
    for i in 1:100
        # The ! means it mutates the input
        # In this case it overwrites `storage` with new random values
        rand!(storage)
        res += sum(storage)
    end
    return res
end


Using the `@btime` macro, we can measure the performace uplift and the number of allocations from each function. You'll find that the function which allocates a new array each loop iteration will be slower and allocate far more memory.

In [ ]:
@btime allocates_alot() samples = 1000;
@btime allocates_once() samples = 1000;

A more subtle, yet very common place where allocations occur is array slices (e.g. `arr[1:10]`). In Julia, these slices allocate a new array by default. We can get around this by telling Julia that we only want to read the data inside the arr. To do this we use an "array view" which can be acheived with the `view(...)` function or the `@views` macro.

In [ ]:
function slices_allocate(x)
    N = Int(length(x) / 2)
    res = sum(x[1:N])
    return res
end

function view_slices(x)
    N = Int(length(x) / 2)
    #The view function takes the array
    # and the indicies you want a view of.
    res = sum(view(x, 1:N))
    return res
end

function view_macro_slices(x)
    N = Int(length(x) / 2)
    # The @views macro tells Julia all array slices in
    # this line should be array views.
    @views res = sum(x[1:N])
    return res
end

Again, we see a large speed increase and the number of allocations drop!

In [ ]:
x = rand(100000)
@btime slices_allocate(x) samples = 1000;
@btime view_slices(x) samples = 1000;
@btime view_macro_slices(x) samples = 1000;

In [ ]:
# Create your own array and use a view to access a slice of that array.

### StaticArrays
Earlier, we mentioned that Julia arrays are heap-allocated because they have variable length (i.e., you can append to them). If you know the length of your array beforehand, you can use the [StaticArrays](https://juliaarrays.github.io/StaticArrays.jl/stable/) library.

StaticArrays has many types you can use, but the main types are `SVector` for 1D arrays, `SMatrix` for 2D arrays, and `SArray` for n-dimensional arrays (all types have equivalent macros e.g., `@SMatrix`).

To initialize a NxN `SMatrix` we must provide the size to the constructor: `SMatrix{N,N}(...)`

In [15]:
m = SMatrix{25,25}(rand(10,10));

A common pattern in atomistic simulation packages is to have an array of `SVectors` which represent atomic properties, like their position. Storing the data for each atom as an SVector can be much faster than using the default Julia `Vector`. This is because the compiler is able to allocate the `SVectors` next to eachother in memory (better locality) whereas with a `Vector` of `Vector` the compiler will not store the data adjacent in memory.

In [ ]:
N_atoms = 1000
static_positions = [SVector{3}(rand(3)) for i in 1:N_atoms];
dynamic_positions = [rand(3) for i in 1:N_atoms];

function distance_matrix(positions)
    N = length(positions)
    dists = zeros(N,N)
    for i in 1:N
        for j in i:N
            dists[i,j] = norm(positions[i] - positions[j])
            dists[j,i] = dists[i,j]
        end
    end
    return dists
end

In [ ]:
println("Dynamic Array Benchmark:")
@benchmark distance_matrix(dynamic_positions)

In [ ]:
println("Static Vector Benchmark:")
@benchmark distance_matrix(static_positions)

Note that `SVector`s etc. are statically sized and immutable. The StaticArrays package also provides statically sized but mutable variants, which are prefixed with `M` instead of `S`, e.g. `MMatrix`.

### Type Stability
Julia will do its best to infer the data types of your variables, but if the compiler cannot infer the type of all your variables this leads to type instability and will slow down your code. We can check type stability with the `@code_warntype` macro. For today's purposes, if this macro returns any red text, then your code has unstable types.

In the example below, the functions are equivalent except one relies on a global variable and the other takes that variable as a parameter. Because of the global variable, the compiler must assume that the variables inside the *unstable* function have `Any` type. Whereas in the type-stable code, the compiler can create a specialized version of the stable function for the specific type you passed into it. This is not the only unstable pattern, so if you really care about speed, check your code with `@code_warntype`.

In [ ]:
global_x = rand(Float64, 1000)
function unstable()
    s = 0
    for val in global_x
      s = s + val
    end
    return s
end

function stable(x)
  s = zero(eltype(x))
  for val in x
     s = s + val
  end
  return s
end

In [ ]:
@code_warntype unstable()

In [ ]:
@code_warntype stable(global_x)

In [ ]:
@btime unstable();

In [ ]:
@btime stable($global_x);